In [1]:
!pip install datasets transformers==4.11.2 sentencepiece

     |████████████████████████████████| 325 kB 14.6 MB/s 
     |████████████████████████████████| 2.9 MB 76.3 MB/s 
     |████████████████████████████████| 1.2 MB 92.3 MB/s 
     |████████████████████████████████| 596 kB 71.6 MB/s 
     |████████████████████████████████| 67 kB 6.4 MB/s 
     |████████████████████████████████| 895 kB 79.6 MB/s 
     |████████████████████████████████| 3.3 MB 70.1 MB/s 
     |████████████████████████████████| 1.1 MB 66.7 MB/s 
     |████████████████████████████████| 212 kB 72.8 MB/s 
     |████████████████████████████████| 134 kB 77.0 MB/s 
     |████████████████████████████████| 127 kB 84.9 MB/s 
     |████████████████████████████████| 94 kB 3.7 MB/s 
     |████████████████████████████████| 144 kB 84.5 MB/s 
     |████████████████████████████████| 271 kB 90.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
  

In [14]:
import time

def get_time():

    return time.perf_counter()

 

def time_since(t, msg=''):

    ts = get_time() - t

    print(f"{msg} took {ts:.3f} seconds.")

    return ts

In [3]:
from datasets import *
from transformers import *
from tokenizers import *
import os
import json

# download and prepare cc_news dataset
dataset = load_dataset("cc_news", split="train")



Generating train split:   0%|          | 0/708241 [00:00<?, ? examples/s]

Dataset cc_news downloaded and prepared to /root/.cache/huggingface/datasets/cc_news/plain_text/1.0.0/ae469e556251e6e7e20a789f93803c7de19d0c4311b6854ab072fecb4e401bd6. Subsequent calls will reuse this data.


In [5]:
dataset.info

DatasetInfo(description='CC-News containing news articles from news sites all over the world The data is available on AWS S3 in the Common Crawl bucket at /crawl-data/CC-NEWS/. This version of the dataset has 708241 articles. It represents a small portion of English  language subset of the CC-News dataset created using news-please(Hamborg et al.,2017) to collect and extract English language portion of CC-News.\n', citation='@InProceedings{Hamborg2017,\n  author     = {Hamborg, Felix and Meuschke, Norman and Breitinger, Corinna and Gipp, Bela},\n  title      = {news-please: A Generic News Crawler and Extractor},\n  year       = {2017},\n  booktitle  = {Proceedings of the 15th International Symposium of Information Science},\n  location   = {Berlin},\n  doi        = {10.5281/zenodo.4120316},\n  pages      = {218--223},\n  month      = {March}\n}\n', homepage='https://commoncrawl.org/2016/10/news-dataset-available/', license='', features={'title': Value(dtype='string', id=None), 'text': V

In [4]:
# split the dataset into training (90%) and testing (10%)
d = dataset.train_test_split(test_size=0.99) # 0.1

In [19]:
type(train_dataset)

datasets.arrow_dataset.Dataset

In [12]:
test_dataset.to_csv('test_news.csv')

Creating CSV from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

642598205

In [8]:
len(d["train"])

7082

In [5]:
# split the dataset into training (90%) and testing (10%)
d2 = dataset.train_test_split(test_size=0.01) # 0.1

In [10]:
len(d2["test"])

7083

In [6]:
for t in d["train"]["text"][:2]:
  print(t)
  print("="*50)

Rodgers did not log any touches from scrimmage but returned three kickoffs for 60 yards in Sunday's 24-21 loss to the Lions.
The versatile veteran didn't see any snaps on offense for the first time since Week 9, as Doug Martin's return from a concussion made Rodgers the odd man out in the backfield. The seven-year veteran has been minimally involved as a running back since the first three games of the season, when he served as the lead back while Martin finished serving his suspension. As such, his fantasy value remains limited to formats that factor in return yards.
SAN ANTONIO, April 3 A prominent wine industry businessman and financial manager pleaded guilty in federal court in San Antonio on Monday to stealing millions of dollars from retired National Basketball Association superstar Tim Duncan.
Charles Banks, 49, faces up to 20 years in prison during his sentencing, due in June. He was indicted by a grand jury of two counts of federal wire fraud on suspicion of bilking the NBA sta

In [7]:
for t in d2["test"]["text"][:2]:
  print(t)
  print("="*50)

Highgrove mattresses consist of more than 60 models ranging from coil springs to the latest pocket latex. All the mattresses are manufactured keeping in mind the difference in shape, size and weight of consumers. A good balance for those ideal sleeping positions can only be achieved with a perfectly balanced mattress. Highgrove maintains a strict rule in their production unit to assure that all their mattresses are produced in accordance with the required balance.
Latex Collection
All natural latex is sourced from Belgium, which is one of the leading producers of the finest quality latex. Under this collection there is Euphoria, Celeste, Aspire and Galaxy. This range is made from bamboo yarn which is stronger, softer and more absorbent then cotton. Its features are anti-bacterial, anti-bug, anti-fungal, eco- friendly, hypoallergenic, firm support, pocket sprung, natural latex, non-turn, firm side support and stretchable fabric.
Pocket Collection
The number of individual pocket springs 

In [8]:
d['train'], d2['test']

(Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
     num_rows: 7082
 }), Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
     num_rows: 7083
 }))

In [ ]:
# if you have your custom dataset 
# dataset = LineByLineTextDataset(
#     tokenizer=tokenizer,
#     file_path="path/to/data.txt",
#     block_size=64,
# )

# or if you have huge custom dataset separated into files
# load the splitted files
# files = ["train1.txt", "train2.txt"] # train3.txt, etc.
# dataset = load_dataset("text", data_files=files, split="train")



In [10]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer 
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train7k.txt")
# save the testing set to test.txt
dataset_to_text(d2["test"], "test7k.txt")

In [12]:
TOKENIZER_TRAINING_FILE = 'train7k.txt' # change to train.txt to train on more data

In [15]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = [TOKENIZER_TRAINING_FILE]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 30_522
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

# initialize the WordPiece tokenizer
tokenizer = BertWordPieceTokenizer()
# train the tokenizer
t=get_time()
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
time_since(t, 'training tokenizer')
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

training tokenizer took 3.553 seconds.


In [16]:
model_path = "pretrained-bert-7k"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

# save the tokenizer  
tokenizer.save_model(model_path)

# dumping some of the tokenizer config to config file, 
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained(model_path)

In [17]:
train_unencoded = d["train"]
test_unencoded = d2["test"]
train_unencoded, test_unencoded

(Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
     num_rows: 7082
 }), Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url'],
     num_rows: 7083
 }))

In [19]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation


# Tokenizing the training data
* Here, make sure you map the right dataset to the encode function (for instance, if you are using a smaller subset)

In [20]:
t = get_time()

# tokenizing the train dataset
train_dataset = train_unencoded.map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = test_unencoded.map(encode, batched=True)

time_since(t, 'tokenizing training and test data')

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as 
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
train_dataset, test_dataset

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

tokenizing training and test data took 9.185 seconds.


(Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
     num_rows: 7082
 }), Dataset({
     features: ['title', 'text', 'domain', 'date', 'description', 'url', 'image_url', 'input_ids', 'token_type_ids', 'attention_mask', 'special_tokens_mask'],
     num_rows: 7083
 }))

# Group text examples into batches

In [23]:
# Main data processing function that will concatenate all texts
# from our dataset and generate chunks of
# max_seq_length.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True, batch_size=2_000,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True, batch_size=2_000,
                                  num_proc=4, desc=f"Grouping texts in chunks of {max_length}")



# Configure model training

In [28]:
# initialize the model with the config
model_config = BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = BertForMaskedLM(config=model_config)

# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2
)

LOGGING_STEPS = 100  # set to 500 for larger models
training_args = TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,      
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps= LOGGING_STEPS,             # evaluate, log and save model checkpoints every 1000 step
    save_steps= LOGGING_STEPS,
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

using `logging_steps` to initialize `eval_steps` to 100
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [29]:
print(f'test dataset has length {len(test_dataset)}')
print(f'train dataset has length {len(train_dataset)}')

test dataset has length 7083
train dataset has length 7082


In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# initialize the trainer and pass everything to it
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


# Start training

In [35]:
t = get_time()
# train the model
trainer.train()
time_since(t, "training")

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: title, description, url, domain, text, special_tokens_mask, date, image_url.
***** Running training *****
  Num examples = 7082
  Num Epochs = 10
  Instantaneous batch size per device = 10
  Total train batch size (w. parallel, distributed & accumulation) = 80
  Gradient Accumulation steps = 8
  Total optimization steps = 880


Step,Training Loss,Validation Loss
500,7.189800,9.008741


The following columns in the evaluation set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: title, description, url, domain, text, special_tokens_mask, date, image_url.
***** Running Evaluation *****
  Num examples = 70825
  Batch size = 64
Saving model checkpoint to pretrained-bert/checkpoint-500
Configuration saved in pretrained-bert/checkpoint-500/config.json
Model weights saved in pretrained-bert/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




3409.537 seconds.


3409.5368782760006

In [31]:
train_dataset.dataset_size

2016418133

In [32]:
train_dataset.size_in_bytes

2861549279

In [ ]:
# when you load from pretrained
# model = BertForMaskedLM.from_pretrained(os.path.join(model_path, "checkpoint-10000"))
# tokenizer = BertTokenizerFast.from_pretrained(model_path)
fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)

# perform predictions
example = "It is known that [MASK] is the capital of Germany"
for prediction in fill_mask(example):
  print(prediction)